In [2]:
import pandas as pd

df = pd.read_csv('data/train_data_categories.csv')
df = df[df['video_id'] != 'b4d70f82038d1d97f1b3ce2a493d12c8'].reset_index(drop=True)

def get_first_level(x):
    tags = []
    for part in x.split(','):
        # tags.append(part.split(':')[0].strip())
        tags.append(part.strip())
    return ', '.join(tags)

def get_second_level(x):
    if x.count(':') > 0:
        return x.split(':')[1].strip()
    else:
        return '-'
    
def get_third_level(x):
    if x.count(':') == 2:
        return x.split(':')[2].strip()
    else:
        return '-'

df['first_level'] = df['tags'].apply(get_first_level)

In [3]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
import re
from tqdm.notebook import tqdm
import os
import json

prompt_template = PromptTemplate(
    input_variables=["title"],
    template=(
        "Классифицируй к какой категории относится описание видео, верни только подходяющую категорию (может быть 1 или несколько). "
        "Напиши только подходящую(ие) категории через ', ' "
        "Транспорт, Книги и литература, Бизнес и финансы, Карьера, Образование, События и достопримечательности, Семья и отношения, Изобразительное искусство, Еда и напитки, Здоровый образ жизни, Хобби и интересы, Дом и сад, Медицина, Фильмы и анимация, Музыка и аудио, Новости и политика, Личные финансы, Животные, Массовая культура, Недвижимость, Религия и духовность, Наука, Покупки, Спорт, Стиль и красота, Информационные технологии, Телевидение, Путешествия, Игры. "
        "Заголовок видео: \"{title}\". "
    ),
)


llm = Ollama(model="llama3", temperature=0)

def process_title_with_llama(title):
    prompt = prompt_template.format(title=title)
    response = llm.invoke(prompt)
    print(response)
    return response

# df = df.iloc[:30]

title_tags = []
for x in tqdm(df['description'].tolist()):
    title_tags.append(process_title_with_llama(x))

df['llm_description_tags'] = title_tags

df['llm_description_tags_list'] = df['llm_description_tags'].apply(lambda x: x.split(', '))
df_llm_description = df['llm_description_tags_list'].str.join('|').str.get_dummies()

  0%|          | 0/1048 [00:00<?, ?it/s]

Транспорт, Массовая культура
Транспорт, Спорт
Семья и отношения, Массовая культура
Транспорт, Спорт
Музыка и аудио, Массовая культура
История и массовая культура, События и достопримечательности
Карьера, Семья и отношения, Массовая культура
Бизнес и финансы, Личные финансы, Массовая культура
История и культура, Путешествия, События и достопримечательности
События и достопримечательности, Путешествия
Образование, Здоровый образ жизни, Медицина
Музыка и аудио, Массовая культура, Путешествия
Категории: Карьера, Образование, Массовая культура, Религия и духовность
Транспорт, Путешествия, Спорт
Семья и отношения, Путешествия, События и достопримечательности
Массовая культура, Фильмы и анимация, Музыка и аудио
Транспорт, Массовая культура, Телевидение
Транспорт, Спорт
Транспорт, Путешествия
Еда и напитки, Хобби и интересы
Транспорт, Массовая культура, Игры, Музыка и аудио, События и достопримечательности
Дом и сад, Карьера, Образование
Искусство и культура, Массовая культура
Транспорт, Массо

In [4]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
import re
from tqdm.notebook import tqdm
import os
import json

prompt_template = PromptTemplate(
    input_variables=["title"],
    template=(
        "Классифицируй к какой категории относится заголовок, верни только подходяющую категорию (может быть 1 или несколько). "
        "Напиши только подходящую(ие) категории через ', ' "
        "Транспорт, Книги и литература, Бизнес и финансы, Карьера, Образование, События и достопримечательности, Семья и отношения, Изобразительное искусство, Еда и напитки, Здоровый образ жизни, Хобби и интересы, Дом и сад, Медицина, Фильмы и анимация, Музыка и аудио, Новости и политика, Личные финансы, Животные, Массовая культура, Недвижимость, Религия и духовность, Наука, Покупки, Спорт, Стиль и красота, Информационные технологии, Телевидение, Путешествия, Игры. "
        "Заголовок видео: \"{title}\". "
    ),
)


llm = Ollama(model="llama3", temperature=0)

def process_title_with_llama(title):
    prompt = prompt_template.format(title=title)
    response = llm.invoke(prompt)
    print(response)
    return response

# df = df.iloc[:30]

title_tags = []
for x in tqdm(df['title'].tolist()):
    title_tags.append(process_title_with_llama(x))

df['llm_title_tags'] = title_tags # df['title'].apply(process_title_with_llama)

df['llm_title_tags_list'] = df['llm_title_tags'].apply(lambda x: x.split(', '))
df_llm = df['llm_title_tags_list'].str.join('|').str.get_dummies()

df_llm_all = pd.concat((df_llm, df_llm_description), axis=1)

  0%|          | 0/1048 [00:00<?, ?it/s]

Транспорт, Массовая культура
Транспорт, Фильмы и анимация
Массовая культура, Фильмы и анимация
Транспорт, Спорт
Музыка и аудио
Массовая культура, Музыка и аудио
Карьера, Образование, Социальные вопросы
Транспорт, Массовая культура
Транспорт, События и достопримечательности, Путешествия
Карьера, Массовая культура
Транспорт, Спорт
Музыка и аудио
Медицина, Массовая культура
Транспорт, Путешествия, Спорт
Категории: Карьера, События и достопримечательности, Массовая культура
Музыка и аудио, Массовая культура
Категории: Карьера, Массовая культура, Телевидение
События и достопримечательности, Спорт
События и достопримечательности, Спорт
Еда и напитки, Кулинария
Карьера, Массовая культура
Категории: Карьера, Образование, Путешествия
Искусство и культура, Изобразительное искусство
Массовая культура, Арт и развлекательные мероприятия
Путешествия
Искусство и культура, Изобразительное искусство
События и достопримечательности, История и архитектура
Транспорт, Путешествия
Искусство и культура, Изоб

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
import re
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Загрузка стоп-слов и пунктуации
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('russian'))
punctuation = set(string.punctuation)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
    text = ' '.join([x for x in text.split() if x not in stop_words])
    return text

# Разделение классов в first_level на списки
df['first_level_list'] = df['first_level'].apply(lambda x: x.split(', '))

# Преобразование текстов меток в мультилейбел формат
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['first_level_list'])

# Очистка текстов
df['cleaned_title'] = df['title'].apply(clean_text)
df['cleaned_description'] = df['description'].apply(clean_text)

# Разделение данных на обучающую и тестовую выборки
train_df, test_df = train_test_split(df, test_size=0.4, random_state=50)

X_train_title = train_df['cleaned_title']
X_train_description = train_df['cleaned_description']
llm_train = df_llm_all.loc[train_df.index]
y_train = y[train_df.index]

X_test_title = test_df['cleaned_title']
X_test_description = test_df['cleaned_description']
llm_test = df_llm_all.loc[test_df.index]
y_test = y[test_df.index]

tfidf_title = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=350,
    min_df=5,
)
X_train_title = tfidf_title.fit_transform(X_train_title).toarray()
X_test_title = tfidf_title.transform(X_test_title).toarray()

tfidf_description = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=150,
    min_df=5,
)
X_train_description = tfidf_description.fit_transform(X_train_description).toarray()
X_test_description = tfidf_description.transform(X_test_description).toarray()

train_features = np.hstack((X_train_title, llm_train.values))
test_features = np.hstack((X_test_title, llm_test.values))

# Создание модели CatBoostClassifier
catboost_model = CatBoostClassifier(
    iterations=2000, 
    learning_rate=0.03,
    depth=4,
    verbose=500,
    random_seed=42,
    eval_metric='AUC',
    early_stopping_rounds=2000,
)

# Функция для обучения OneVsRestClassifier с CatBoost и eval_set
def fit_ovr_with_eval_set(X_train, y_train, X_valid, y_valid):
    n_classes = y_train.shape[1]
    estimators = []
    
    for i in range(n_classes):
        if y_train.sum(axis=0)[i] < 3 or y_test.sum(axis=0)[i] == 0:
            estimators.append(None)
            continue

        print(f"Training model for class {i + 1}/{n_classes}")
        model = CatBoostClassifier(
            iterations=3500, 
            learning_rate=0.03,
            depth=3,
            verbose=100,
            random_seed=42,
            # eval_metric='AUC',
            early_stopping_rounds=1000,
        )
        model.fit(
            X_train, y_train[:, i],
            eval_set=(X_valid, y_valid[:, i])
        )
        estimators.append(model)
    
    return estimators

# Обучение модели
ovr_estimators = fit_ovr_with_eval_set(train_features, y_train, test_features, y_test)

# Предсказание на тестовых данных
y_pred = np.zeros(y_test.shape)

for i, model in enumerate(ovr_estimators):
    if model is not None:
        y_pred[:, i] = model.predict_proba(test_features)[:, 1]


y_pred = (y_pred > 0.3).astype(np.uint8)

# Вычисление accuracy и F1-меры для мультилейбел данных
accuracy = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 micro: {f1_micro:.4f}')
print(f'F1 macro: {f1_macro:.4f}')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dmitrii/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/dmitrii/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training model for class 8/155
0:	learn: 0.6392074	test: 0.6400803	best: 0.6400803 (0)	total: 643us	remaining: 2.25s
100:	learn: 0.0017290	test: 0.0162254	best: 0.0158870 (90)	total: 43.7ms	remaining: 1.47s
200:	learn: 0.0005349	test: 0.0203083	best: 0.0158870 (90)	total: 86.3ms	remaining: 1.42s
300:	learn: 0.0003078	test: 0.0233651	best: 0.0158870 (90)	total: 126ms	remaining: 1.34s
400:	learn: 0.0002137	test: 0.0255574	best: 0.0158870 (90)	total: 167ms	remaining: 1.29s
500:	learn: 0.0001657	test: 0.0269707	best: 0.0158870 (90)	total: 206ms	remaining: 1.23s
600:	learn: 0.0001357	test: 0.0280572	best: 0.0158870 (90)	total: 245ms	remaining: 1.18s
700:	learn: 0.0001149	test: 0.0289905	best: 0.0158870 (90)	total: 283ms	remaining: 1.13s
800:	learn: 0.0001001	test: 0.0298112	best: 0.0158870 (90)	total: 322ms	remaining: 1.08s
900:	learn: 0.0000886	test: 0.0305319	best: 0.0158870 (90)	total: 361ms	remaining: 1.04s
1000:	learn: 0.0000784	test: 0.0312010	best: 0.0158870 (90)	total: 400ms	remaini

/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [84]:
# Предсказание на тестовых данных
y_pred_proba = np.zeros(y_test.shape)

for i, model in enumerate(ovr_estimators):
    if model is not None:
        y_pred_proba[:, i] = model.predict_proba(test_features)[:, 1]


y_pred = (y_pred_proba > 0.2).astype(np.uint8)

# Вычисление accuracy и F1-меры для мультилейбел данных
accuracy = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 micro: {f1_micro:.4f}')
print(f'F1 macro: {f1_macro:.4f}')

Accuracy: 0.4405
F1 micro: 0.6304
F1 macro: 0.1284


/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [85]:
bad_ids = y_pred.sum(axis=1)==0
for i in range(len(y_pred)):
    if bad_ids[i] and y_pred_proba.max(axis=1)[i] > 0:
        print(i)
        y_pred[i, y_pred_proba.argmax(axis=1)[i]] = 1

0
22
31
37
40
50
54
77
80
85
99
101
129
133
134
145
155
169
174
186
188
194
199
200
210
216
230
241
245
248
263
268
277
278
289
290
291
316
318
321
325
331
340
349
356
358
362
387
396
399
414


In [86]:
# Вычисление accuracy и F1-меры для мультилейбел данных
accuracy = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 micro: {f1_micro:.4f}')
print(f'F1 macro: {f1_macro:.4f}')

Accuracy: 0.4595
F1 micro: 0.6318
F1 macro: 0.1293


/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [89]:
y_pred_labels = mlb.inverse_transform(y_pred)
#y_pred_formatted = [", ".join(labels) for labels in y_pred_labels]

test_df = df.loc[test_df.index]
test_df['predicted_tags'] = y_pred_labels

In [90]:
import pandas as pd
import argparse
import ast
import numpy as np

def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    return final_tag_list


def find_iou_for_sample_submission(df, pred_df):
    df["tags_new"] = df["tags"].apply(lambda l: l.split(', '))
    df["tags_split"] = df["tags_new"].apply(lambda l: split_tags(l))

    # pred_df["predicted_tags"] = pred_df["predicted_tags"].apply(ast.literal_eval)
    pred_df["predicted_tags_split"] = pred_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in df.iterrows():
        predicted_tags = pred_df[pred_df["video_id"]==row["video_id"]]["predicted_tags_split"].values[0]
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter

sample_df = pd.read_csv('sample_submission.csv')
sample_df['predicted_tags'] = sample_df['predicted_tags'].apply(lambda x: [x])

final_score = find_iou_for_sample_submission(test_df, sample_df)
print(final_score)


final_score = find_iou_for_sample_submission(test_df, test_df)
print(final_score)

0.07615646258503396
0.6260317460317459
